In [1]:
# importamos las librerías necesarias para trabajar.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#Otros 
import warnings
warnings.filterwarnings('ignore')

In [2]:
#SK-Learn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

#Scalers
#https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing
from sklearn.preprocessing import StandardScaler,MinMaxScaler
#Modelos Lineales
# https://scikit-learn.org/stable/modules/linear_model.html
from sklearn.linear_model import LinearRegression , Ridge
from sklearn.svm import SVR
#Metricas
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

### Abro el dataset generado

In [3]:
df_confirmados = pd.read_feather('./data/pivot_data')
df_confirmados.head(60)

,index,argentino,asistencia_respiratoria,confirmados,cuidado_intensivo,edad,fallecidos,mayor_65,mayor_65_internacion,presentan_sintomas,privado,publico,internacion,sexo_F,sexo_M
0,2020-03-01,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0
1,2020-03-02,1,0,1,0,43.000000,0,0,0,1,1,0,1,0,1
2,2020-03-03,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0
3,2020-03-04,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0
4,2020-03-05,5,1,5,1,50.600000,1,2,1,5,3,2,3,1,4
5,2020-03-06,1,0,1,0,68.000000,0,1,1,1,1,0,1,0,1
6,2020-03-07,1,0,1,0,71.000000,0,1,0,1,0,1,0,0,1
7,2020-03-08,1,0,1,0,52.000000,0,0,0,1,0,1,0,0,1
8,2020-03-09,1,0,1,0,24.000000,0,0,0,1,0,1,0,0,1
9,2020-03-10,3,0,3,1,53.666667,1,2,2,3,2,1,2,0,3


In [4]:
df_confirmados.index = df_confirmados['index']
df_confirmados.drop('index', axis=1, inplace=True)

In [5]:
df_confirmados.tail()

,argentino,asistencia_respiratoria,confirmados,cuidado_intensivo,edad,fallecidos,mayor_65,mayor_65_internacion,presentan_sintomas,privado,publico,internacion,sexo_F,sexo_M
index,,,,,,,,,,,,,,
2020-09-26,676,7,692,9,44.251445,10,121,29,359,393,299,53,348,344
2020-09-27,556,2,567,6,40.890212,7,76,21,335,307,260,45,274,293
2020-09-28,764,3,785,6,41.391614,9,113,26,431,480,305,53,399,386
2020-09-29,891,4,904,4,42.636799,7,135,25,454,531,373,57,449,455
2020-09-30,829,3,852,9,42.082355,11,133,28,452,512,340,58,425,427


In [6]:
df_confirmados.shape

(214, 14)

### Probamos distintos modelos de regresion


#### Divido el dataset en train-test (sin usar split, para mantener la temporalidad)

In [7]:
#selecciono las variables a utilizar
#selecciono el primer 70% de los datos para entrenar el modelo

# x_train =  np.array(df_confirmados[["asistencia_respiratoria","confirmados","cuidado_intensivo","presentan_sintomas","internacion"]].iloc[0:150,:])
# #x_train =  np.array(df_confirmados[["cuidado_intensivo","asistencia_respiratoria","presentan_sintomas","internacion"]].iloc[0:150,:])
# y_train =  np.array(df_confirmados[["fallecidos"]].iloc[0:150]) 

# #me quedo con el 30% restante para testear
# x_test = np.array(df_confirmados[["asistencia_respiratoria","confirmados","cuidado_intensivo","presentan_sintomas","internacion"]].iloc[150:215,:])
# #x_test = np.array(df_confirmados[["asistencia_respiratoria","cuidado_intensivo","presentan_sintomas","internacion"]].iloc[150:215,:])
# y_test = np.array(df_confirmados[["fallecidos"]].iloc[150:2215,:]) 



# #PRUEBO OTRA PARTICION
# #selecciono el primer 70% de los datos para entrenar el modelo
df_confirmados= df_confirmados.loc[df_confirmados.index >= pd.Timestamp(year=2020, month=5, day=1), :] #desde mayo
df_confirmados

#entreno con el 70% de datos desde mayo y testeo con el 30% restante (desde mediados de agosto hasta fin de sept)
# x_train =  np.array(df_confirmados[["asistencia_respiratoria","confirmados","cuidado_intensivo","mayor_65","presentan_sintomas","internacion"]].iloc[0:108,:])
x_train =  np.array(df_confirmados[["asistencia_respiratoria","confirmados","cuidado_intensivo","mayor_65","internacion"]].iloc[0:108,:])
y_train =  np.array(df_confirmados[["fallecidos"]].iloc[0:108]) 

# x_test =  np.array(df_confirmados[["asistencia_respiratoria","confirmados","cuidado_intensivo","mayor_65","presentan_sintomas","internacion"]].iloc[108:154,:])
x_test =  np.array(df_confirmados[["asistencia_respiratoria","confirmados","cuidado_intensivo","mayor_65","internacion"]].iloc[108:154,:])
y_test =  np.array(df_confirmados[["fallecidos"]].iloc[108:154])


# #PRUEBO OTRA PARTICION
# #selecciono el primer 80% de los datos para entrenar el modelo  
# df_confirmados= df_confirmados.loc[df_confirmados.index >= pd.Timestamp(year=2020, month=5, day=1), :] #desde mayo
# df_confirmados

# #entreno con el 85% de datos desde mayo y testeo con el 15% restante (desde mediados de agosto hasta fin de sept)
# x_train =  np.array(df_confirmados[["asistencia_respiratoria","confirmados","cuidado_intensivo","mayor_65","internacion"]].iloc[0:131,:])
# y_train =  np.array(df_confirmados[["fallecidos"]].iloc[0:131]) 

# x_test =  np.array(df_confirmados[["asistencia_respiratoria","confirmados","cuidado_intensivo","mayor_65","internacion"]].iloc[131:154,:])
# y_test =  np.array(df_confirmados[["fallecidos"]].iloc[131:154])

In [8]:
df_confirmados

,argentino,asistencia_respiratoria,confirmados,cuidado_intensivo,edad,fallecidos,mayor_65,mayor_65_internacion,presentan_sintomas,privado,publico,internacion,sexo_F,sexo_M
index,,,,,,,,,,,,,,
2020-05-01,49,0,52,0,37.729167,3,5,4,45,13,39,27,28,24
2020-05-02,47,1,49,2,32.061224,0,4,2,41,16,33,25,24,25
2020-05-03,40,1,41,2,33.024390,2,3,2,35,11,30,18,21,20
2020-05-04,65,1,68,7,35.950980,3,6,5,63,23,45,33,34,34
2020-05-05,101,2,102,4,37.343137,4,7,7,91,24,78,57,59,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-26,676,7,692,9,44.251445,10,121,29,359,393,299,53,348,344
2020-09-27,556,2,567,6,40.890212,7,76,21,335,307,260,45,274,293
2020-09-28,764,3,785,6,41.391614,9,113,26,431,480,305,53,399,386


## Pruebo con el TimeSeriesSplit

In [ ]:
# x_train =  np.array(df_confirmados[["asistencia_respiratoria","confirmados","cuidado_intensivo","mayor_65","internacion"]])
# y_train =  np.array(df_confirmados[["fallecidos"]]) 
# x_test =  np.array(df_confirmados[["asistencia_respiratoria","confirmados","cuidado_intensivo","mayor_65","internacion"]])
# y_test =  np.array(df_confirmados[["fallecidos"]])

In [9]:
# Vamos a crear un dataframe para guardar los resultados de cada estimador
results_df = pd.DataFrame(columns=['Model','Features','R2','MSE'])
results_df

,Model,Features,R2,MSE


In [10]:
# Definimos el escalador 
# # auto scaling train- set (mean = 0, std = 1)
scaler = preprocessing.StandardScaler()
scaler

StandardScaler()

In [11]:
# Transformamos los x_train y x_test
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [12]:
# Aqui estamos agregando las features polinomicas LUEGO de escalar los datos
grad_poly = 2
poly = PolynomialFeatures(grad_poly)
x_train_poly_scaled = poly.fit_transform(x_train_scaled)
x_test_poly_scaled = poly.transform(x_test_scaled)

## Linear Reg  (c/ TimeSeriesSplit - CV)
#### Linear Features

In [13]:
tscv = TimeSeriesSplit(n_splits=5)
#Definimos el estimador
est = LinearRegression()
#Definimos diccionario con hyperparametros
parameters = {'fit_intercept' : [False, True]}
#Definimos el objeto grid search con los parametros anteriores
gs = GridSearchCV(est, param_grid=parameters, cv=tscv, scoring="neg_mean_squared_error")

In [14]:
#Entrenamos
gs.fit(x_train_scaled, y_train)

# Mostramos el mejor estimador, sus hyperparametros seleccionados y su resultado
print(gs.best_estimator_, "\n")
print(gs.best_params_, "\n")
print(gs.best_score_, "\n")

LinearRegression() 

{'fit_intercept': True} 

-30.660122948423286 



In [15]:
# Usamos x_test (del train- test split) para predecir con el mejor estimador
linear_prediction = gs.best_estimator_.predict(x_test_scaled)

#Calculamos las metricas R2, MSE y MAE
lineal_linear_r2 = r2_score(y_true=y_test, y_pred=linear_prediction)
lineal_linear_mse = mean_squared_error(y_true=y_test, y_pred=linear_prediction)

results_df = results_df.append({'Model':'Linear',
                                'Features':'Linear',
                                'R2':lineal_linear_r2,
                                'MSE':lineal_linear_mse},ignore_index=True)

results_df

,Model,Features,R2,MSE
0,Linear,Linear,0.787333,21.070325


#### Poly Features

In [16]:
gs = GridSearchCV(est, param_grid=parameters,refit=True, cv=tscv, scoring="neg_mean_squared_error")
#Entrenamos
gs.fit(x_train_poly_scaled, y_train)
# Mostramos el mejor estimador, sus hyperparametros seleccionados y su resultado
print(gs.best_estimator_, "\n")
print(gs.best_params_, "\n")
print(gs.best_score_, "\n")

LinearRegression() 

{'fit_intercept': True} 

-843.5452662830481 



In [17]:
# Predecimos para el set polinomico
poly_linear_prediction = gs.best_estimator_.predict(x_test_poly_scaled)

# Calculamos las metricas R2,MSE y MAE
poly_linear_r2 = r2_score(y_true=y_test , y_pred=poly_linear_prediction)
poly_linear_mse = mean_squared_error(y_true=y_test , y_pred=poly_linear_prediction)

In [18]:
results_df = results_df.append({'Model':'Linear',
                                'Features':'Poly',
                                'R2':poly_linear_r2,
                                'MSE':poly_linear_mse},ignore_index=True)
results_df

,Model,Features,R2,MSE
0,Linear,Linear,0.787333,21.070325
1,Linear,Poly,0.710553,28.677416


### Support Vector Regression (C/ TimeSeriesSplit)
#### Linear Features

In [19]:
svr = SVR()
#Definimos diccionario con hyperparametros
parameters = parameters = {'C' : [1000,1500,2000,3000,5000],
              'epsilon' : [0.001, 0.01, 0.1,1, 10,100],
             'gamma':[0.001, 0.01, 0.1,1, 10,100]}
#Definimos el objeto grid search con los parametros anteriores
gs = GridSearchCV(estimator=svr, param_grid=parameters, cv=tscv, scoring="neg_mean_squared_error")

In [20]:
# fiteo + Prediccion + Metricas
gs.fit(x_train_scaled, y_train)

# Mostramos el mejor estimador, sus hyperparametros seleccionados y su resultado
print(gs.best_estimator_, "\n")
print(gs.best_params_, "\n")
print(gs.best_score_, "\n")


SVR(C=5000, epsilon=1, gamma=0.001) 

{'C': 5000, 'epsilon': 1, 'gamma': 0.001} 

-31.750253423062873 



In [21]:
#Predicciones + Metricas
svr_linear_prediction = gs.predict(x_test_scaled)

# Prediccion + Metricas

svr_linear_r2 = r2_score(y_true=y_test, y_pred=svr_linear_prediction)
svr_linear_mse = mean_squared_error(y_true=y_test, y_pred=svr_linear_prediction)

In [22]:
results_df = results_df.append({'Model':'SVR',
                                'Features':'Linear',
                                'R2':svr_linear_r2,
                                'MSE':svr_linear_mse},ignore_index=True)
results_df

,Model,Features,R2,MSE
0,Linear,Linear,0.787333,21.070325
1,Linear,Poly,0.710553,28.677416
2,SVR,Linear,0.795169,20.293990


#### Poly features

In [23]:
svr = SVR()
#Definimos diccionario con hyperparametros
parameters = parameters = {'C' : [1000,1500,2000,3000,5000],
              'epsilon' : [0.001, 0.01, 0.1,1, 10,100],
             'gamma':[0.001, 0.01, 0.1,1, 10,100]}
#Definimos el objeto grid search con los parametros anteriores
gs = GridSearchCV(estimator=svr, param_grid=parameters, cv=tscv, scoring="neg_mean_squared_error")

In [24]:
gs.fit(x_train_poly_scaled, y_train)
# Mostramos el mejor estimador, sus hyperparametros seleccionados y su resultado
print(gs.best_estimator_, "\n")
print(gs.best_params_, "\n")
print(gs.best_score_, "\n")


SVR(C=5000, epsilon=0.01, gamma=0.001) 

{'C': 5000, 'epsilon': 0.01, 'gamma': 0.001} 

-68.03070755176641 



In [25]:
poly_svr_prediction = gs.predict(x_test_poly_scaled)

poly_svr_r2 = r2_score(y_true=y_test, y_pred=poly_svr_prediction)
poly_svr_mse = mean_squared_error(y_true=y_test, y_pred=poly_svr_prediction)

In [26]:
#Guardamos
results_df = results_df.append({'Model':'SVR',
                                'Features':'Poly',
                                'R2':poly_svr_r2,
                                'MSE':poly_svr_mse},ignore_index=True)
results_df

,Model,Features,R2,MSE
0,Linear,Linear,0.787333,21.070325
1,Linear,Poly,0.710553,28.677416
2,SVR,Linear,0.795169,20.293990
3,SVR,Poly,0.712737,28.461059


### Ridge Regression (C/ TimeSeriesSplit)
#### Linear Features

In [27]:
# Ahora cambiamos el estimador, usaremos Ridge Regression
ridge = Ridge(alpha=1.7)

In [28]:
#Definimos diccionario con hyperparametros
# Lista del parametro lambda (llamado 'alpha' en el GridSearch )
lambdas = [0.001,0.005,0.01,0.02,0.05,0.1,0.2,0.3,0.4,0.5,1]
# Juntamos el diccionario de parametros
parameters = {'alpha': lambdas}
#Definimos el objeto grid search con los parametros anteriores
gs = GridSearchCV(estimator=ridge, param_grid=parameters, cv=tscv, scoring="neg_mean_squared_error")

In [29]:
# Entrenamos (con datos escalados)
gs.fit(x_train_scaled, y_train)

# Mostramos el mejor estimador, sus hyperparametros seleccionados y su resultado
print(gs.best_estimator_, "\n")
print(gs.best_params_, "\n")
print(gs.best_score_, "\n")

#Predicciones + Metricas
ridge_linear_prediction = gs.predict(x_test_scaled)
ridge_linear_r2 = r2_score(y_true=y_test, y_pred=ridge_linear_prediction)
ridge_linear_mse = mean_squared_error(y_true=y_test, y_pred=ridge_linear_prediction)


Ridge(alpha=0.001) 

{'alpha': 0.001} 

-30.662016678967355 



In [30]:
#Guardamos resultados
results_df = results_df.append({'Model':'Ridge',
                                'Features':'Linear',
                                'R2':ridge_linear_r2,
                                'MSE':ridge_linear_mse},ignore_index=True)
results_df

,Model,Features,R2,MSE
0,Linear,Linear,0.787333,21.070325
1,Linear,Poly,0.710553,28.677416
2,SVR,Linear,0.795169,20.293990
3,SVR,Poly,0.712737,28.461059
4,Ridge,Linear,0.787337,21.069942


#### Poly features


In [31]:
# Entrenamos (con datos escalados)
gs.fit(x_train_poly_scaled, y_train)
#Predicciones + Metricas
ridge_poly_prediction = gs.predict(x_test_poly_scaled)

# Mostramos el mejor estimador, sus hyperparametros seleccionados y su resultado
print(gs.best_estimator_, "\n")
print(gs.best_params_, "\n")
print(gs.best_score_, "\n")


ridge_poly_r2 = r2_score(y_true=y_test, y_pred=ridge_poly_prediction)
ridge_poly_mse = mean_squared_error(y_true=y_test, y_pred=ridge_poly_prediction)


Ridge(alpha=0.2) 

{'alpha': 0.2} 

-56.971432959791095 



In [32]:
#Guardamos resultados
results_df = results_df.append({'Model':'Ridge',
                                'Features':'Poly',
                                'R2':ridge_poly_r2,
                                'MSE':ridge_poly_mse},ignore_index=True)
results_df

,Model,Features,R2,MSE
0,Linear,Linear,0.787333,21.070325
1,Linear,Poly,0.710553,28.677416
2,SVR,Linear,0.795169,20.293990
3,SVR,Poly,0.712737,28.461059
4,Ridge,Linear,0.787337,21.069942
5,Ridge,Poly,0.736166,26.139716


### Linear Regression

#### Linear Features 

In [ ]:
#Definimos el estimador
lr = LinearRegression()
lr.fit(x_train_scaled, y_train)

# Usamos x_test (del train- test split) para predecir con el mejor estimador
linear_prediction = lr.predict(x_test_scaled)

In [ ]:
#Calculamos las metricas R2, MSE y MAE
lineal_linear_r2 = r2_score(y_true=y_test, y_pred=linear_prediction)
lineal_linear_mse = mean_squared_error(y_true=y_test, y_pred=linear_prediction)
lineal_linear_mae = mean_absolute_error(y_true=y_test, y_pred=linear_prediction)

In [ ]:
#Vamos a plotear Predictions vs True values
plt.scatter(linear_prediction, y_test)
plt.plot(np.arange(0,100),np.arange(0,100), 'r', label='Identity') #Linea identidad (Predictions = True Values)
plt.xlabel('Predictions')
plt.ylabel('True Values')
plt.xlim((10,70))
plt.ylim((0,60))
plt.legend()
print(f'R2 score: {lineal_linear_r2:.6f}')
print(f'MAE: {lineal_linear_mae:.6f}')
print(f'MSE: {lineal_linear_mse:.6f}')


In [ ]:
#Guardamos los resultados
results_df = results_df.append({'Model':'Linear',
                                'Features':'Linear',
                                'R2':lineal_linear_r2,
                                'MSE':lineal_linear_mse,
                                'MAE':lineal_linear_mae},ignore_index=True)

In [ ]:
results_df

### Polynomial Features

In [ ]:
lr = LinearRegression()
lr.fit(x_train_poly_scaled, y_train)
# Predecimos para el set polinomico
poly_linear_prediction = lr.predict(x_test_poly_scaled)

# Calculamos las metricas R2,MSE y MAE
poly_linear_r2 = r2_score(y_true=y_test , y_pred=poly_linear_prediction)
poly_linear_mse = mean_squared_error(y_true=y_test , y_pred=poly_linear_prediction)
poly_linear_mae = mean_absolute_error(y_true=y_test , y_pred=poly_linear_prediction)

In [ ]:
#Vamos a plotear Predictions vs True values del set polinomico
plt.scatter(poly_linear_prediction, y_test)
plt.plot(np.arange(0,100),np.arange(0,100), 'r', label='Identity')
plt.xlabel('Predictions')
plt.ylabel('True Values')
plt.xlim((10,70))
plt.ylim((0,60))
plt.legend()
print(f'R2 score: {poly_linear_r2:.6f}')
print(f'MAE: {poly_linear_mae:.6f}')
print(f'MSE: {poly_linear_mse:.6f}')

In [ ]:
results_df = results_df.append({'Model':'Linear',
                                'Features':'Poly',
                                'R2':poly_linear_r2,
                                'MSE':poly_linear_mse,
                                'MAE':poly_linear_mae},ignore_index=True)
results_df

### Support Vector Regression

In [ ]:
# Ahora cambiamos el estimador, usaremos SVR
svr = SVR(max_iter=25000)
svr

#### Linear Features

In [ ]:
# fiteo + Prediccion + Metricas
svr.fit(x_train_scaled, y_train)
svr_linear_prediction = svr.predict(x_test_scaled)

svr_linear_r2 = r2_score(y_true=y_test, y_pred=svr_linear_prediction)
svr_linear_mse = mean_squared_error(y_true=y_test, y_pred=svr_linear_prediction)
svr_linear_mae = mean_absolute_error(y_true=y_test, y_pred=svr_linear_prediction)

In [ ]:
#Vamos a plotear Predictions vs True values del set polinomico
plt.scatter(svr_linear_prediction, y_test)
plt.plot(np.arange(0,100),np.arange(0,100), 'r', label='Identity')
plt.xlabel('Predictions')
plt.ylabel('True Values')
plt.xlim((10,70))
plt.ylim((0,60))
plt.legend()
print(f'R2 score: {svr_linear_r2:.6f}')
print(f'MAE: {svr_linear_mae:.6f}')
print(f'MSE: {svr_linear_mse:.6f}')

In [ ]:
results_df = results_df.append({'Model':'SVR',
                                'Features':'Linear',
                                'R2':svr_linear_r2,
                                'MSE':svr_linear_mse,
                                'MAE':svr_linear_mae},ignore_index=True)
results_df

#### Features Polinomicas

In [ ]:
# fiteo + Prediccion + Metricas
svr.fit(x_train_poly_scaled, y_train)
poly_svr_prediction = svr.predict(x_test_poly_scaled)

poly_svr_r2 = r2_score(y_true=y_test, y_pred=poly_svr_prediction)
poly_svr_mse = mean_squared_error(y_true=y_test, y_pred=poly_svr_prediction)
poly_svr_mae = mean_absolute_error(y_true=y_test, y_pred=poly_svr_prediction)

In [ ]:
#Plot
plt.scatter(poly_svr_prediction, y_test)
plt.plot(np.arange(0,100),np.arange(0,100), 'r', label='Identity')
plt.xlabel('Predictions')
plt.ylabel('True Values')
plt.xlim((0,70))
plt.ylim((0,60))
plt.legend()
print(f'R2 score: {poly_svr_r2:.6f}')
print(f'MAE: {poly_svr_mae:.6f}')
print(f'MSE: {poly_svr_mse:.6f}')

In [ ]:
#Guardamos
results_df = results_df.append({'Model':'SVR',
                                'Features':'Poly',
                                'R2':poly_svr_r2,
                                'MSE':poly_svr_mse,
                                'MAE':poly_svr_mae},ignore_index=True)
results_df

### Ridge Regression


#### Linear Features

In [ ]:
# Ahora cambiamos el estimador, usaremos Ridge Regression
ridge = Ridge(alpha=1.7)

In [ ]:
# Entrenamos (con datos escalados)
ridge.fit(x_train_scaled, y_train)
#Predicciones + Metricas
ridge_linear_prediction = ridge.predict(x_test_scaled)
ridge_linear_r2 = r2_score(y_true=y_test, y_pred=ridge_linear_prediction)
ridge_linear_mse = mean_squared_error(y_true=y_test, y_pred=ridge_linear_prediction)
ridge_linear_mae = mean_absolute_error(y_true=y_test, y_pred=ridge_linear_prediction)

In [ ]:
# Plots 
plt.scatter(ridge_linear_prediction, y_test)
plt.plot(np.arange(0,100),np.arange(0,100), 'r', label='Identity')
plt.xlabel('Predictions')
plt.ylabel('True Values')
plt.xlim((0,70))
plt.ylim((0,60))
plt.legend()
print(f'R2 score: {ridge_linear_r2:.6f}')
print(f'MAE: {ridge_linear_mae:.6f}')
print(f'MSE: {ridge_linear_mse:.6f}')

In [ ]:
#Guardamos resultados
results_df = results_df.append({'Model':'Ridge',
                                'Features':'Linear',
                                'R2':ridge_linear_r2,
                                'MSE':ridge_linear_mse,
                                'MAE':ridge_linear_mae},ignore_index=True)
results_df

#### Polynomial Features 

In [ ]:
# Fiteo + Prediccion + Metricas
ridge.fit(x_train_poly_scaled, y_train)
poly_ridge_prediction = ridge.predict(x_test_poly_scaled)

poly_ridge_r2 = r2_score(y_true=y_test, y_pred=poly_ridge_prediction)
poly_ridge_mse = mean_squared_error(y_true=y_test, y_pred=poly_ridge_prediction)
poly_ridge_mae = mean_absolute_error(y_true=y_test, y_pred=poly_ridge_prediction)

In [ ]:
# Plots
plt.scatter(poly_ridge_prediction, y_test)
plt.plot(np.arange(0,100),np.arange(0,100), 'r', label='Identity')
plt.xlabel('Predictions')
plt.ylabel('True Values')
plt.xlim((0,70))
plt.ylim((0,60))
plt.legend()
print(f'R2 score: {poly_ridge_r2:.6f}')
print(f'MAE: {poly_ridge_mae:.6f}')
print(f'MSE: {poly_ridge_mse:.6f}')

In [ ]:
# Guardamos
results_df = results_df.append({'Model':'Ridge',
                                'Features':'Poly',
                                'R2':poly_ridge_r2,
                                'MSE':poly_ridge_mse,
                                'MAE':poly_ridge_mae},ignore_index=True)
results_df

### KNN Regression

#### Linear Features

In [ ]:
knr = KNeighborsRegressor()

In [ ]:
# Entrenamos (con datos escalados)
knr.fit(x_train_scaled, y_train)
#Predicciones + Metricas
knr_linear_prediction = knr.predict(x_test_scaled)
knr_linear_r2 = r2_score(y_true=y_test, y_pred=knr_linear_prediction)
knr_linear_mse = mean_squared_error(y_true=y_test, y_pred=knr_linear_prediction)
knr_linear_mae = mean_absolute_error(y_true=y_test, y_pred=knr_linear_prediction)

In [ ]:
# Plots 
plt.scatter(knr_linear_prediction, y_test)
plt.plot(np.arange(0,100),np.arange(0,100), 'r', label='Identity')
plt.xlabel('Predictions')
plt.ylabel('True Values')
plt.xlim((0,70))
plt.ylim((0,60))
plt.legend()
print(f'R2 score: {knr_linear_r2:.6f}')
print(f'MAE: {knr_linear_mae:.6f}')
print(f'MSE: {knr_linear_mse:.6f}')

In [ ]:
#Guardamos resultados
results_df = results_df.append({'Model':'KNR',
                                'Features':'Linear',
                                'R2':knr_linear_r2,
                                'MSE':knr_linear_mse,
                                'MAE':knr_linear_mae},ignore_index=True)
results_df

#### Polynomial Features

In [ ]:
# Fiteo + Prediccion + Metricas
knr.fit(x_train_poly_scaled, y_train)
poly_knr_prediction = knr.predict(x_test_poly_scaled)

poly_knr_r2 = r2_score(y_true=y_test, y_pred=poly_knr_prediction)
poly_knr_mse = mean_squared_error(y_true=y_test, y_pred=poly_knr_prediction)
poly_knr_mae = mean_absolute_error(y_true=y_test, y_pred=poly_knr_prediction)

In [ ]:
# Plots
plt.scatter(poly_knr_prediction, y_test)
plt.plot(np.arange(0,100),np.arange(0,100), 'r', label='Identity')
plt.xlabel('Predictions')
plt.ylabel('True Values')
plt.xlim((0,70))
plt.ylim((0,60))
plt.legend()
print(f'R2 score: {poly_knr_r2:.6f}')
print(f'MAE: {poly_knr_mae:.6f}')
print(f'MSE: {poly_knr_mse:.6f}')

In [ ]:
# Guardamos
results_df = results_df.append({'Model':'KNR',
                                'Features':'Poly',
                                'R2':poly_knr_r2,
                                'MSE':poly_knr_mse,
                                'MAE':poly_knr_mae},ignore_index=True)
results_df

### Abro el dataset de movilidad de google

In [ ]:
df_mov = pd.read_excel('./data/dataset_movilidad_CABA.xlsx',parse_dates=['date'])
df_mov

In [ ]:
df_mov = df_mov.drop(['country_region_code','metro_area','iso_3166_2_code','census_fips_code',], axis=1)

In [ ]:
df_mov=df_mov.rename(columns={'retail_and_recreation_percent_change_from_baseline':'tiendas_y_ocio',
            'grocery_and_pharmacy_percent_change_from_baseline':'supermercados_y_farmacias',
            'parks_percent_change_from_baseline':'parques','transit_stations_percent_change_from_baseline':'estaciones_de_transporte',
            'workplaces_percent_change_from_baseline':'lugares_de_trabajo','residential_percent_change_from_baseline':'zonas_residenciales'})

In [ ]:
df_mov

In [ ]:
df_mov_prom = df_mov.groupby('date').mean()
df_mov_prom

#### Recorto periodo desde 01-03-20 hasta 30-09-20

In [ ]:
df_mov_prom = df_mov_prom.loc[(df_mov_prom.index >= pd.Timestamp(year=2020, month=3, day=1)) & 
                               (df_mov_prom.index <= pd.Timestamp(year=2020, month=9, day=30)), :]#desde marzo    
df_mov_prom.head(123+62)

### Probamos modelos de regresion con estas nuevas features y confirmados

In [ ]:
#train del 70% (desde marzo) y testeo con el 30% (desde finales de julio)
#x_train =  np.array(df_mov_prom[["tiendas_y_ocio",'estaciones_de_transporte','lugares_de_trabajo']].iloc[0:151,:])
# x_train =  np.array(df_mov_prom[["tiendas_y_ocio",'estaciones_de_transporte','lugares_de_trabajo']])
# y_train =  np.array(df_confirmados[["confirmados"]])

# x_test = np.array(df_mov_prom[["tiendas_y_ocio",'estaciones_de_transporte','lugares_de_trabajo']])
# y_test =  np.array(df_confirmados[["confirmados"]])
########################################################################################################################
#train del desde marzo hasta agosto y testeo con las primeras 2 semanas de agosto
# x_train =  np.array(df_mov_prom[["tiendas_y_ocio",'estaciones_de_transporte','lugares_de_trabajo']].iloc[123:185,:])
# y_train =  np.array(df_confirmados[["confirmados"]].iloc[123:185])

# x_test =  pd.DataFrame(df_mov_prom[["tiendas_y_ocio",'estaciones_de_transporte',
#         'lugares_de_trabajo']]).loc[(df_mov_prom.index >= pd.Timestamp(year=2020, month=8, day=1)) & 
#                                (df_mov_prom.index <= pd.Timestamp(year=2020, month=8, day=15)), :]
# y_train =  pd.DataFrame(df_confirmados[["confirmados"]]).loc[(df_confirmados.index >= pd.Timestamp(year=2020, month=3, day=1)) & 
#                                (df_confirmados.index <= pd.Timestamp(year=2020, month=8, day=15)), :]

# x_test =  np.array(df_mov_prom[["tiendas_y_ocio",'estaciones_de_transporte','lugares_de_trabajo']].iloc[185:215,:])
# y_test =  np.array(df_confirmados[["confirmados"]].iloc[185:215])

########################################################################################################################
#entreno y testeo con todo el dataset

x_train =  np.array(df_mov_prom[["tiendas_y_ocio",'estaciones_de_transporte','lugares_de_trabajo']])
y_train =  np.array(df_confirmados[["confirmados"]])
                    
x_test =  np.array(df_mov_prom[["tiendas_y_ocio",'estaciones_de_transporte','lugares_de_trabajo']])
y_test =  np.array(df_confirmados[["confirmados"]])

In [ ]:
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

In [ ]:
plt.scatter(df_mov_prom.estaciones_de_transporte, df_confirmados.confirmados)

In [ ]:
# Vamos a crear un dataframe para guardar los resultados de cada estimador
results_df = pd.DataFrame(columns=['Model','Features','R2','MSE'])
results_df

In [ ]:
# Definimos el escalador 
# # auto scaling train- set (mean = 0, std = 1)
scaler = preprocessing.StandardScaler()
scaler

In [ ]:
# Transformamos los x_train y x_test
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Aqui estamos agregando las features polinomicas LUEGO de escalar los datos
grad_poly = 2
poly = PolynomialFeatures(grad_poly)

x_train_poly_scaled = poly.fit_transform(x_train_scaled)
x_test_poly_scaled = poly.transform(x_test_scaled)

### Linear Regression

#### Linear Features

In [ ]:
#Definimos el estimador
lr = LinearRegression()
lr.fit(x_train_scaled, y_train)

# Usamos x_test (del train- test split) para predecir con el mejor estimador
linear_prediction = lr.predict(x_test_scaled)

In [ ]:
#Calculamos las metricas R2, MSE y MAE
lineal_linear_r2 = r2_score(y_true=y_test, y_pred=linear_prediction)
lineal_linear_mse = mean_squared_error(y_true=y_test, y_pred=linear_prediction)

In [ ]:
#Guardamos los resultados
results_df = results_df.append({'Model':'Linear',
                                'Features':'Linear',
                                'R2':lineal_linear_r2,
                                'MSE':lineal_linear_mse},ignore_index=True)
results_df

In [ ]:
#Vamos a plotear Predictions vs True values del set polinomico
plt.scatter(linear_prediction, y_test)
# plt.plot(linear_prediction,y_test, 'r', label='Identity')
plt.xlabel('Predictions')
plt.ylabel('True Values')
# plt.xlim((10,70))
# plt.ylim((0,60))
# plt.legend()
print(f'R2 score: {lineal_linear_r2:.6f}')
print(f'MSE: {lineal_linear_mse:.6f}')



#### Polynomial Features

In [ ]:
lr = LinearRegression()
lr.fit(x_train_poly_scaled, y_train)

poly_linear_prediction = lr.predict(x_test_poly_scaled)

# Calculamos las metricas R2,MSE y MAE
poly_linear_r2 = r2_score(y_true=y_test , y_pred=poly_linear_prediction)
poly_linear_mse = mean_squared_error(y_true=y_test , y_pred=poly_linear_prediction)

In [ ]:
results_df = results_df.append({'Model':'Linear',
                                'Features':'Poly',
                                'R2':poly_linear_r2,
                                'MSE':poly_linear_mse},ignore_index=True)
results_df

### Support Vector Regression

#### Linear Features

In [ ]:
# Ahora cambiamos el estimador, usaremos SVR
svr = SVR(max_iter=25000)


In [ ]:
# fiteo + Prediccion + Metricas
svr.fit(x_train_scaled, y_train)
svr_linear_prediction = svr.predict(x_test_scaled)

svr_linear_r2 = r2_score(y_true=y_test, y_pred=svr_linear_prediction)
svr_linear_mse = mean_squared_error(y_true=y_test, y_pred=svr_linear_prediction)

In [ ]:
results_df = results_df.append({'Model':'SVR',
                                'Features':'Linear',
                                'R2':svr_linear_r2,
                                'MSE':svr_linear_mse},ignore_index=True)
results_df

#### Polynomial features

In [ ]:
# fiteo + Prediccion + Metricas
svr.fit(x_train_poly_scaled, y_train)
poly_svr_prediction = svr.predict(x_test_poly_scaled)

poly_svr_r2 = r2_score(y_true=y_test, y_pred=poly_svr_prediction)
poly_svr_mse = mean_squared_error(y_true=y_test, y_pred=poly_svr_prediction)

In [ ]:
#Guardamos
results_df = results_df.append({'Model':'SVR',
                                'Features':'Poly',
                                'R2':poly_svr_r2,
                                'MSE':poly_svr_mse},ignore_index=True)
results_df

### Ridge Regression

#### Linear Features

In [ ]:
# Ahora cambiamos el estimador, usaremos Ridge Regression
ridge = Ridge(alpha=1.5)

In [ ]:
# Entrenamos (con datos escalados)
ridge.fit(x_train_scaled, y_train)
#Predicciones + Metricas
ridge_linear_prediction = ridge.predict(x_test_scaled)
ridge_linear_r2 = r2_score(y_true=y_test, y_pred=ridge_linear_prediction)
ridge_linear_mse = mean_squared_error(y_true=y_test, y_pred=ridge_linear_prediction)

In [ ]:
#Guardamos resultados
results_df = results_df.append({'Model':'Ridge',
                                'Features':'Linear',
                                'R2':ridge_linear_r2,
                                'MSE':ridge_linear_mse},ignore_index=True)
results_df

#### Polynomial Features

In [ ]:
# Fiteo + Prediccion + Metricas
ridge.fit(x_train_poly_scaled, y_train)
poly_ridge_prediction = ridge.predict(x_test_poly_scaled)

poly_ridge_r2 = r2_score(y_true=y_test, y_pred=poly_ridge_prediction)
poly_ridge_mse = mean_squared_error(y_true=y_test, y_pred=poly_ridge_prediction)

In [ ]:
# Guardamos
results_df = results_df.append({'Model':'Ridge',
                                'Features':'Poly',
                                'R2':poly_ridge_r2,
                                'MSE':poly_ridge_mse},ignore_index=True)
results_df